# High-level training libraries

Writing the training loop is my least favourite thing about PyTorch.

Keras is great here!
```python
model.compile(optimizer, criterion, metrics=["accuracy", "f1"])
model.fit(X, y, epochs=10)
```


## Init, helpers, utils, ...

In [1]:
%matplotlib inline

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision

DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
DEVICE

device(type='cuda', index=0)

In [3]:
from pprint import pprint
import random

import matplotlib.pyplot as plt
import numpy as np
from IPython.core.debugger import set_trace

In [4]:
from utils import (
    get_model,
    get_data,
)
from my_train_helper import get_trainable

In [5]:
train_dl, val_dl = get_data()

Loading data from ../data/raw/dogscats/sample/train.
Loading data from ../data/raw/dogscats/sample/valid.


# The Contenders
## [Ignite](https://github.com/pytorch/ignite)
> Ignite is a high-level library to help with training neural networks in PyTorch.
> - ignite helps you write compact but full-featured training loops in a few lines of code
> - you get a training loop with metrics, early-stopping, model checkpointing and other features without the boilerplate


## [TNT](https://github.com/pytorch/tnt)
> TNT is a library providing powerful dataloading, logging and visualization utlities for Python. It is closely intergrated with PyTorch and is designed to enable rapid iteration with any model or training regimen.
> [...]
> The project was inspired by TorchNet, and legend says that it stood for “TorchNetTwo”


## [Skorch](https://github.com/dnouri/skorch)
> A scikit-learn compatible neural network library that wraps PyTorch.


## [Poutyne](https://poutyne.org/) (formally PyToune)
> Poutyne is a Keras-like framework for PyTorch and handles much of the boilerplating code needed to train neural networks.


## [fast.ai](https://docs.fast.ai/)
> The fastai library simplifies training fast and accurate neural nets using modern best practices. It's based on research in to deep learning best practices undertaken at fast.ai, including "out of the box" support for vision, text, tabular, and collab (collaborative filtering) models.


## [MagNet](https://github.com/MagNet-DL/magnet)
> MagNet is a high-level Deep Learning API, wrapped around PyTorch.
> 
> It was developed with the aim of reducing boilerplate code and writing Deep Learning architectures with more grace.

```python
model = nn.Sequential(mn.Conv(32), *mn.Conv() * 2, mn.Linear(10, act=None))
```

## "The Fun of Reinvention"
Clearly, there must be a better way! Write your own lib (but don't use it) :D

# Ignite Demo

In [6]:
import ignite
from ignite.metrics import (
    Accuracy,
    Loss,
    Precision,
)
from ignite.engine import (
    create_supervised_evaluator,
    create_supervised_trainer,
    Events,
)

In [7]:
# model, loss, optimizer
model = get_model()
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(
    get_trainable(model.parameters()),
    lr=0.001,
    momentum=.9,
)

# trainer and evaluator
trainer = create_supervised_trainer(model, optimizer, criterion, device=DEVICE)
evaluator = create_supervised_evaluator(
    model,
    metrics={
        "accuracy": Accuracy(),
        "loss": Loss(criterion),
        "precision": Precision(),
    },
    device=DEVICE,
)

In [8]:
@trainer.on(Events.ITERATION_COMPLETED)
def log_training_loss(engine):
    print(
        f"Epoch[{engine.state.epoch}] "
        f"Batch[{engine.state.iteration}] "
        f"Loss: {engine.state.output:.2f}"
    )

# trainer.run(train_dl, max_epochs=1)

In [9]:
@trainer.on(Events.EPOCH_COMPLETED)
def log_training_results(trainer):
    evaluator.run(train_dl)
    metrics = evaluator.state.metrics
    print(
        f"Training Results   - Epoch: {trainer.state.epoch}  "
        f"accuracy: {metrics['accuracy']:.2f} "
        f"loss: {metrics['loss']:.2f} "
        f"prec: {metrics['precision'].cpu()}"
    )


@trainer.on(Events.EPOCH_COMPLETED)
def log_validation_results(engine):
    evaluator.run(val_dl)
    metrics = evaluator.state.metrics
    print(
        f"Validation Results - Epoch: {trainer.state.epoch}  "
        f"accuracy: {metrics['accuracy']:.2f} "
        f"loss: {metrics['loss']:.2f} "
        f"prec: {metrics['precision'].cpu()}"
    )


trainer.run(train_dl, max_epochs=1)

Epoch[1] Batch[1] Loss: 0.55
Epoch[1] Batch[2] Loss: 1.11
Epoch[1] Batch[3] Loss: 0.63
Epoch[1] Batch[4] Loss: 1.12
Epoch[1] Batch[5] Loss: 0.87
Epoch[1] Batch[6] Loss: 0.69
Epoch[1] Batch[7] Loss: 0.62
Epoch[1] Batch[8] Loss: 0.41
Training Results   - Epoch: 1  accuracy: 0.88 loss: 0.51 prec: tensor([0.8750, 0.8750], dtype=torch.float64)
Validation Results - Epoch: 1  accuracy: 0.88 loss: 0.39 prec: tensor([0.8000, 1.0000], dtype=torch.float64)


# Exercise
- Try out PyToune.
- Try out another library.

In [10]:
import poutyne
from poutyne.framework import Model

In [11]:
train_dl, val_dl = get_data()

module = get_model()
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(
    get_trainable(module.parameters()),
    lr=0.001,
    momentum=.9,
)

Loading data from ../data/raw/dogscats/sample/train.
Loading data from ../data/raw/dogscats/sample/valid.


In [12]:
# you need two lines of code
# model = Model(...)
# ...

# Visualization with Tensorboard
- https://www.tensorflow.org/programmers_guide/summaries_and_tensorboard
- https://pytorch.org/docs/stable/tensorboard.html

![](img/tensorboardx_demo.gif)

Make sure tensorboard and tensorboardx are installed:
```
conda install tensorboard
```


Start tensorboard:
```
cd notebooks
tensorboard --logdir=tf_log
```

In [13]:
rm -rf tf_log/*

In [14]:
ls tf_log/

In [15]:
from torch.utils.tensorboard import SummaryWriter

summary_writer = SummaryWriter(log_dir=f"tf_log/demo_{random.randint(0, 1000)}")

In [16]:
ls tf_log

demo_348/


In [17]:
# write some scalars
for i in range(10):
    summary_writer.add_scalar("training/loss", np.random.rand(), i)
    summary_writer.add_scalar("validation/loss", np.random.rand() + .1, i)

Then visit http://localhost:6006

In [18]:
# Visualize the graph/network
X, _ = next(iter(train_dl))
summary_writer.add_graph(model.cpu(), X.cpu())

Then visit http://localhost:6006

## Use tensorboard with ignite

In [19]:
import datetime


# new SummaryWriter for new experiment
now = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
summary_writer = SummaryWriter(log_dir=f"tf_log/exp_ignite_{now}")

# Basic setup: model, loss, optimizer
model = get_model()
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(get_trainable(model.parameters()), lr=0.0001, momentum=.9)

# trainer and evaluator
trainer = create_supervised_trainer(model, optimizer, criterion, device=DEVICE)
evaluator = create_supervised_evaluator(
    model,
    metrics={"accuracy": Accuracy(), "loss": Loss(criterion)},
    device=DEVICE,
)

@trainer.on(Events.EPOCH_COMPLETED)
def log_training_results(trainer):
    evaluator.run(train_dl)
    metrics = evaluator.state.metrics
    epoch = trainer.state.epoch
    summary_writer.add_scalar("training/accuracy", metrics['accuracy'], epoch)
    summary_writer.add_scalar("training/loss", metrics['loss'], epoch)


@trainer.on(Events.EPOCH_COMPLETED)
def log_validation_results(engine):
    evaluator.run(val_dl)
    metrics = evaluator.state.metrics
    epoch = trainer.state.epoch
    summary_writer.add_scalar("validation/accuracy", metrics['accuracy'], epoch)
    summary_writer.add_scalar("validation/loss", metrics['loss'], epoch)
    print(metrics['accuracy'])


trainer.run(train_dl, 1)

0.25


# Visualization with Visdom
https://github.com/facebookresearch/visdom
![](img/visdom.png)

## Exercise
- Use visdom with ignite.
- Use visdom with any other library.